In [35]:
ip_type = 4 # Set to 4 or 6 for IPv4 or IPv6

In [36]:
# Function to check if the header is valid and extract qr, question count, and answer count
def check_header_get_vals (packet):

    # Check if packet has at least 24 bytes (header length)
    if len(packet) < 24:
        return -1

    # Extract the flags and answer count
    flags = int(packet[4:8], 16)
    question_count = int(packet[8:12], 16)
    answer_count = int(packet[12:16], 16)

    if question_count != 1 or answer_count > 1: # Only 1 question and at most 1 answer
        return -1
    
    qr = (flags & 0x8000) >> 15
    opcode = (flags & 0x7800) >> 11
    aa = (flags & 0x0400) >> 10
    tc = (flags & 0x0200) >> 9
    rd = (flags & 0x0100) >> 8
    ra = (flags & 0x0080) >> 7
    z = (flags & 0x0070) >> 4
    rcode = (flags & 0x000F)

    if qr == 1 and answer_count == 0: # Response packet with no answers
        return -1
    
    if qr == 0 and answer_count > 0: # Query packet with answers
        return -1

    if opcode > 2: # opcode = 0, 1, 2
        print("Invalid data")
        return -1
    
    if z != 0: # z = 0
        return -1
    
    if rcode > 5: # rcode = 0, 1, 2, 3, 4, 5
        return -1
    
    return qr

In [37]:
# Function to extract domain name from the packet
def extract_domain_name(packet, start_index):
    domain_name = ""
    index = start_index
    length = int(packet[index:index + 2], 16)

    if length & 0xC0 == 0xC0:  # Message Compression has been used
        offset = (int(packet[index:index + 4], 16) & 0x3FFF) * 2
        domain_name, _ = extract_domain_name(packet, offset)
        return (domain_name, index + 4)
    
    if length & 0xC0 != 0x00:  # Invalid length - first 2 bits should be 00 or 11
        return ("Invalid data", -1)
    
    if len(packet) < index + 2 + length * 2: # Packet is too short
        return ("Invalid data", -1)

    while length != 0: # 0-length label indicates end of domain name
        if len(domain_name) > 0:
            domain_name += "."
        domain_name += (bytes.fromhex(packet[index + 2: index + 2 + length * 2])).decode('utf-8')

        index += 2 + length * 2
        length = int(packet[index:index + 2], 16)
        if len(packet) < index + 2 + length * 2: # Packet is too short
            return ("Invalid data", -1)

    return (domain_name, index + 2)

In [38]:
# Converts hex code to IP address
def hex_to_ip_address(hex_ip):
    ip_address = ""
    for i in range(0, 2 * ip_type, 2):
        ip_address += str(int(hex_ip[i:i + 2], 16))
        if i < 2 * ip_type - 2:
            ip_address += "."
    return ip_address


# Function to extract IP address from the packet
def extract_ip_address(packet, start_index):
    if len(packet) < start_index + 2 * ip_type: # Packet is too short
        return "Invalid data"
    return hex_to_ip_address(packet[start_index: start_index + 2 * ip_type])

In [39]:
def parse_dns_packet(packet):
    
    qr = check_header_get_vals(packet)

    if qr == -1: # Invalid header
        print("Invalid data")
        return
    
    domain_name, ptr = extract_domain_name(packet, 24)
    if domain_name == "Invalid data":
        print("Invalid data")
        return
    
    # Check if it's a query or response packet
    if qr == 0:  # Query packet
        print("Packet Type: Query")
        print(f"Domain Name: {domain_name}")

    else:  # Response packet
        
        ptr += 8  # Skip the qtype and qclass fields

        answer_domain_name, ptr = extract_domain_name(packet, ptr)
        if answer_domain_name != domain_name:
            print("Invalid data")
            return
        
        ptr += 20 # Skip the type, class, ttl, and rdlength fields

        ip_address = extract_ip_address(packet, ptr)

        if ip_address == "Invalid data":
            print("Invalid data")
            return
        
        print("Packet Type: Response")
        print(f"Domain Name: {domain_name}")
        print(f"IP Address: {ip_address}")

In [40]:
packet1 = "4c76010000010000000000010c74696d65736f66696e6469610a696e64696174696d657303636f6d00000100010000290200000000000000"
parse_dns_packet(packet1)

Packet Type: Query
Domain Name: timesofindia.indiatimes.com


In [41]:
packet2 = "629f8180000100010000000103637365046969746d02616302696e0000010001c00c0001000100001f9600040a0608020000290200000000000000"
parse_dns_packet(packet2)

Packet Type: Response
Domain Name: cse.iitm.ac.in
IP Address: 10.6.8.2


In [42]:
packet3 = "00c88b57ec40ec2e98e9046b08004500004b527e40004011802f0a2a52f10a1800c2e36400350037875137920100000100000000000106636c69656e740764726f70626f7803636f6d00000100010000290200000000000000"
parse_dns_packet(packet3)

Invalid data


In [43]:
packet4 = "ec2e98e9046b34e894fa3f5e08004500006b030400003e11f7c4c0a80001c0a80068003581c300577fa37baf818000010001000000010f7a2d7034322d696e7374616772616d046331307209696e7374616772616d03636f6d0000010001c00c000100010000000c00049df017ae0000290200000000000000"
parse_dns_packet(packet4)

Invalid data


In [44]:
packet5 = "a0208180000100010000000105666f6e74730a676f6f676c656170697303636f6d0000010001c00c00010001000000ae00048efab64a0000290200000000000000"
parse_dns_packet(packet5)

Packet Type: Response
Domain Name: fonts.googleapis.com
IP Address: 142.250.182.74
